<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/GrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

def calculate_average_growth_rate(data, start_year, end_year, column):
    start_value = data.loc[data['year'] == start_year, column].iloc[0]
    end_value = data.loc[data['year'] == end_year, column].iloc[0]
    years = end_year - start_year
    avg_growth_rate = (end_value / start_value) ** (1 / years) - 1
    return avg_growth_rate

# Load data
url = 'https://dataverse.nl/api/access/datafile/354098'
pwt1001 = pd.read_stata(url)

# Filter and select relevant columns
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'country', 'rgdpna', 'rnna', 'emp', 'avh', 'labsh', 'hc', 'rtfpna']]
data = data.loc[(data['year'] >= 1975) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['gdp_per_worker'] = data['rgdpna'] / data['emp']
data['capital_per_worker'] = data['rnna'] / data['emp']
data['human_capital_per_worker'] = data['hc']
data['hours_per_worker'] = data['avh']

# Calculate average growth rates for each country
results = []
for country in countries:
    country_data = data[data['country'] == country]
    alpha = country_data['alpha'].mean()
    gdp_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'gdp_per_worker')
    capital_deepening = calculate_average_growth_rate(country_data, 1975, 2019, 'capital_per_worker') * alpha
    hc_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'human_capital_per_worker')
    hours_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'hours_per_worker')
    labor_growth = (1 - alpha) * (hc_growth + hours_growth)
    tfp_growth = gdp_growth - capital_deepening - labor_growth
    tfp_share = tfp_growth / gdp_growth
    capital_share = capital_deepening / gdp_growth
    labor_share = labor_growth / gdp_growth
    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepening': capital_deepening,
        'Labor Growth': labor_growth,
        'TFP Share': tfp_share,
        'Capital Share': capital_share,
        'Labor Share': labor_share
    })

# Create a DataFrame from the results
growth_accounting_table = pd.DataFrame(results)

# Calculate the average values for each column
average_values = {
    'Country': 'Average',
    'Growth Rate': growth_accounting_table['Growth Rate'].mean(),
    'TFP Growth': growth_accounting_table['TFP Growth'].mean(),
    'Capital Deepening': growth_accounting_table['Capital Deepening'].mean(),
    'Labor Growth': growth_accounting_table['Labor Growth'].mean(),
    'TFP Share': growth_accounting_table['TFP Share'].mean(),
    'Capital Share': growth_accounting_table['Capital Share'].mean(),
    'Labor Share': growth_accounting_table['Labor Share'].mean()
}

# Convert the average_values dictionary to a DataFrame
average_values_df = pd.DataFrame([average_values])

# Concatenate the average_values_df with the growth_accounting_table
growth_accounting_table = pd.concat([growth_accounting_table, average_values_df], ignore_index=True)

# Display the updated table
print(growth_accounting_table.to_string(index=False, float_format='{:.2%}'.format))

       Country  Growth Rate  TFP Growth  Capital Deepening  Labor Growth  TFP Share  Capital Share  Labor Share
     Australia        1.19%       0.69%              0.43%         0.07%     58.07%         36.37%        5.55%
       Austria        1.46%       0.59%              0.79%         0.08%     40.56%         53.97%        5.47%
       Belgium        1.39%       0.50%              0.69%         0.20%     35.99%         49.55%       14.47%
        Canada        0.93%       0.19%              0.55%         0.20%     20.25%         58.66%       21.09%
       Denmark        1.50%       0.79%              0.61%         0.11%     52.22%         40.34%        7.44%
       Finland        1.89%       0.89%              0.82%         0.18%     46.88%         43.49%        9.63%
        France        1.36%       0.71%              0.57%         0.09%     51.73%         41.47%        6.80%
       Germany        1.47%       1.00%              0.59%        -0.12%     67.70%         40.14%      